In [4]:
import googlemaps
# working with gmaps
with open('api.txt') as f:
    API_KEY = f.readline()
    f.close
gmaps = googlemaps.Client(key = API_KEY)

jakarta_coord = (-6.175252901681506, 106.82713921415032)

AttributeError: 'Client' object has no attribute 'figure'